# 2.0: Understanding the `config` file, the `run_*` scripts, & support scripts

이번 워크샵은 다섯 과정을 거쳐서 최종 `decoding` 단계에 도달하게 됩니다. 
 - run_prepare_data: 데이터 처리
 - run_feature_extraction: 음성 파일에서 특성 자질 추출
 - run_train_phones: 음성 모델 학습
 - run_compile_graph: 데이터 통합
 - run_test: decoding
 
그리고 각각 과정에서 필요한 논항들은 `kaldi_config.json` 파일 안에 들어있는 설정값을 이용합니다. 

먼저 `config` 파일의 설정값들이 어떻게 `hyperparameter`로 각각의 단계에 해당하는 스크립트에 전달되는지 확인하겠습니다. 

## `kaldi_config.json`

In [ ]:
cat kaldi_config.json | grep -A15 run_prepare_data
echo ...
cat kaldi_config.json | grep -A15 run_train_phones

**Note:** `path`는 `절대경로`로 입력되어야 합니다. 

### `integer` v. `string` values

이 `json` 파일은 `python` 모듈로 파싱되기 때문에, `string` 값들은 `""` 안에 입력되어야 합니다. 숫자값들은 `""` 없이 입력되어도 괜찮습니다. 

### `boolean` values

`논리값`은 `shell` 스크립트 형태로 입력되어야 하므로, 소문자 `true` 혹은 `false`로 입력됩니다. 문자열이 아니기 때문에 `""`는 필요하지 않습니다. 

### `null` values

특정 설정값이 필요 없는 경우에는 `null`을 입력하여 사용할 수 있습니다. 문자열이 아니기 때문에 `""`는 필요하지 않습니다. 

### `non-vanilla hyperparameters`

모든 스크립트에는 최소 하나의 `non-vanilla hyperparameters`라는 항목이 있습니다. `kaldi` 스크립트는 입력할 수 있는 `flag`의 갯수가 매우 많기 때문에, 모든 파라미터를 `config` 파일에 포함시키는 것은 그리 좋은 선택이 아닙니다. 그러므로 꼭 필요한 부분들만 `config` 파일에 포함시켰고, 나머지 파라미터를 조정하고 싶은 경우에는 아래와 같은 방법으로 `config` 파일을 수정하여 조정할 수 있게 하였습니다.  `config` `key`:

```
"non_vanilla_train_deltas_hyperparameters": {
    "flag": "-s",
    "value": "--num_iters 5 --beam 20"
...
```

## `run_*.sh`

`run_*` 파일의 첫 부분들은 다음의 정보를 담고 있습니다. 
 - 스크립트의 목적
 - 스크립트의 논항구조
 - 스크립트의 결과물


In [ ]:
head -n30 run_prepare_data.sh

각각의 `run_*.sh` 스크립트는 `kaldi config` 파일 하나를 논항으로 받습니다. `run_*.sh` 스크립트에서 맨 처음 하는 일은 필요한 논항을 `kaldi config` 파일에서 찾아서 읽어오는 것입니다. 

## `kaldi` script structure

`run_*.sh` 스크립트는 `shell` 스크립트로 불러와서 작업이 이루어지며, 각각의 스크립트는 `egs/INSTRUCTIONAL` 아래에 저장되어 있습니다. `egs/INSTRUCTIONAL` 아래에 위치한 각 폴더들이 담고 있는 내용은 다음과 같습니다. 

 - `utils`: 파일을 다루는데 필요한 스크립트들
 - `local`: 사용하고 있는 코퍼스를 다루는데 필요한 스크립트
 - `steps`: `ASR` 실행 단계에서 필요한 스크립트

## `path.sh` and `cmd.sh`

마지막으로 `path.sh`와 `cmd.sh` 파일이 남았습니다. 

`path.sh` 스크립트는 단순히 `C++` 코드가 위치한 `kaldi src` 디렉토리의 경로를 담고 있는 파일입니다. `shell` 스크립트 시작 부분에 이 스크립트를 읽어오게 되면, 필요한 `C++` 코드들을 경로를 설정하지 않고도 사용할 수 있기 때문에 유용한 파일입니다. 

`kaldi`는 또한 다양한 `병렬처리`를 지원합니다([이 곳](http://kaldi-asr.org/doc/queue.html)을 참고하세요). 이번 경우에는 단순히 하나의 머신에서 사용하는 `run.pl` 파일을 이용할 것입니다. `cmd.sh`는 병렬처리를 위한 값들을 담고 있습니다. 지금은 간단한 데이터를 다루기 때문에 포함된 내용 또한 단순합니다.

**Note:** 본인의 시스템 사양과 맞지 않는 경우 해당 값을 변경하여야 합니다. 


In [1]:
cat cmd.sh

#!/bin/bash

# set the global variable MEM as the maximum amount of memory you want to utilize (in gigabytes)

MEM="--mem 10G"

export train_cmd="run.pl ${MEM}"
export decode_cmd="run.pl ${MEM}"
export mkgraph_cmd="run.pl ${MEM}"


`16G` 메모리를 기준으로 `10G` 정도는 메모리 값으로는 충분한 세팅값입니다. 본인 사양에 맞게 바꾸어주시면 되겠습니다. 